<a href="https://colab.research.google.com/github/heywassup7/Git-Tutorial/blob/master/LSTM_%EB%AA%A8%EB%8D%B8_%EC%A0%81%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from sklearn.model_selection import train_test_split

In [19]:
# CSV 파일 가져오기

# 코랩 사용 시 아래 두 줄 주석 해제
# from google.colab import files
# uploaded = files.upload()

csv_df = pd.read_csv('dataset.csv')
csv_df

,Unnamed: 0,Date,환율,금리,y,ROE,영업이익률,부채비율,EPS,순이익률,BPS,PER,PBR,종가
0,0,2013-01-01,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
1,1,2013-01-02,1062.500000,2.75,31520.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
2,2,2013-01-03,1062.599976,2.75,30860.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
3,3,2013-01-04,1061.900024,2.75,30500.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
4,4,2013-01-07,1033.099976,2.75,30400.0,20.384758,16.084911,42.701448,197841.0,13.325641,973898.18780,0.138697,0.028175,27440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,3647,2022-12-18,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0
3648,3648,2022-12-24,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0
3649,3649,2022-12-25,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0
3650,3650,2022-12-31,1260.260010,3.25,43500.0,16.424694,14.352127,26.405922,8057.0,18.414395,51356.84057,6.863597,1.076780,55300.0


In [20]:
csv_df = pd.read_csv('samsung.csv')
csv_df

,Unnamed: 0,ROE,영업이익률,부채비율,EPS,순이익률,BPS,종가,PER,PBR,PER.1,PBR.1,PER.2,PBR.2
0,20221231,16.42,14.35,26.41,8057.0,18.41,51356.84,55300.0,6.86,1.08,6.86,1.08,6.86,1.08
1,20211231,13.58,18.47,39.92,5777.0,14.27,44137.30,78300.0,13.55,1.77,13.55,1.77,13.55,1.77
2,20201231,9.71,15.20,37.07,3841.0,11.15,40136.44,81000.0,21.09,2.02,21.09,2.02,21.09,2.02
3,20191231,8.51,12.05,34.12,3166.0,9.44,38285.32,55800.0,17.62,1.46,17.62,1.46,17.62,1.46
4,20181231,18.20,24.16,36.97,6461.0,18.19,36097.38,38700.0,5.99,1.07,5.99,1.07,5.99,1.07
5,20171231,19.47,22.39,40.68,5997.0,17.61,30490.74,50960.0,8.50,1.67,8.50,1.67,8.50,1.67
6,20161231,11.72,14.49,35.87,3159.0,11.26,26822.48,36040.0,11.41,1.34,11.41,1.34,11.41,1.34
7,20151231,10.25,13.16,35.25,126305.0,9.50,1186567.57,25200.0,0.20,0.02,0.20,0.02,0.20,0.02
8,20141231,13.73,12.14,37.08,153105.0,11.35,1100057.63,26540.0,0.17,0.02,0.17,0.02,0.17,0.02
9,20131231,20.38,16.08,42.70,197841.0,13.33,973898.19,27440.0,0.14,0.03,0.14,0.03,0.14,0.03


In [21]:
# 날짜 인덱스가 없음 일단 임시로
csv_df = csv_df.rename(columns={'Unnamed: 0':"날짜"})
csv_df

,날짜,ROE,영업이익률,부채비율,EPS,순이익률,BPS,종가,PER,PBR,PER.1,PBR.1,PER.2,PBR.2
0,20221231,16.42,14.35,26.41,8057.0,18.41,51356.84,55300.0,6.86,1.08,6.86,1.08,6.86,1.08
1,20211231,13.58,18.47,39.92,5777.0,14.27,44137.30,78300.0,13.55,1.77,13.55,1.77,13.55,1.77
2,20201231,9.71,15.20,37.07,3841.0,11.15,40136.44,81000.0,21.09,2.02,21.09,2.02,21.09,2.02
3,20191231,8.51,12.05,34.12,3166.0,9.44,38285.32,55800.0,17.62,1.46,17.62,1.46,17.62,1.46
4,20181231,18.20,24.16,36.97,6461.0,18.19,36097.38,38700.0,5.99,1.07,5.99,1.07,5.99,1.07
5,20171231,19.47,22.39,40.68,5997.0,17.61,30490.74,50960.0,8.50,1.67,8.50,1.67,8.50,1.67
6,20161231,11.72,14.49,35.87,3159.0,11.26,26822.48,36040.0,11.41,1.34,11.41,1.34,11.41,1.34
7,20151231,10.25,13.16,35.25,126305.0,9.50,1186567.57,25200.0,0.20,0.02,0.20,0.02,0.20,0.02
8,20141231,13.73,12.14,37.08,153105.0,11.35,1100057.63,26540.0,0.17,0.02,0.17,0.02,0.17,0.02
9,20131231,20.38,16.08,42.70,197841.0,13.33,973898.19,27440.0,0.14,0.03,0.14,0.03,0.14,0.03


In [22]:
csv_df = csv_df.drop(['PER.1','PBR.1','PER.2','PBR.2'], axis=1, inplace = False)
csv_df

,날짜,ROE,영업이익률,부채비율,EPS,순이익률,BPS,종가,PER,PBR
0,20221231,16.42,14.35,26.41,8057.0,18.41,51356.84,55300.0,6.86,1.08
1,20211231,13.58,18.47,39.92,5777.0,14.27,44137.30,78300.0,13.55,1.77
2,20201231,9.71,15.20,37.07,3841.0,11.15,40136.44,81000.0,21.09,2.02
3,20191231,8.51,12.05,34.12,3166.0,9.44,38285.32,55800.0,17.62,1.46
4,20181231,18.20,24.16,36.97,6461.0,18.19,36097.38,38700.0,5.99,1.07
5,20171231,19.47,22.39,40.68,5997.0,17.61,30490.74,50960.0,8.50,1.67
6,20161231,11.72,14.49,35.87,3159.0,11.26,26822.48,36040.0,11.41,1.34
7,20151231,10.25,13.16,35.25,126305.0,9.50,1186567.57,25200.0,0.20,0.02
8,20141231,13.73,12.14,37.08,153105.0,11.35,1100057.63,26540.0,0.17,0.02
9,20131231,20.38,16.08,42.70,197841.0,13.33,973898.19,27440.0,0.14,0.03


In [23]:
#날짜 빼고 정규화

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scale_cols = ['ROE','영업이익률','부채비율','EPS','순이익률','BPS','종가','PER','PBR']

#정규화 수행
scaled_df = scaler.fit_transform(csv_df[scale_cols]) #정규화된 데이터는 넘파이 형태
scaled_df = pd.DataFrame(scaled_df, columns = scale_cols) #Pandas DataFrame 형태로 변경

#테스트
scaled_df

,ROE,영업이익률,부채비율,EPS,순이익률,BPS,종가,PER,PBR
0,0.666386,0.189926,0.000000,0.025159,1.000000,0.021155,0.539427,0.320764,0.530
1,0.427127,0.530140,0.829343,0.013448,0.538462,0.014930,0.951613,0.640095,0.875
2,0.101095,0.260116,0.654389,0.003503,0.190635,0.011480,1.000000,1.000000,1.000
3,0.000000,0.000000,0.473297,0.000036,0.000000,0.009884,0.548387,0.834368,0.720
4,0.816344,1.000000,0.648250,0.016961,0.975474,0.007997,0.241935,0.279236,0.525
5,0.923336,0.853840,0.875998,0.014578,0.910814,0.003163,0.461649,0.399045,0.825
6,0.270430,0.201486,0.580724,0.000000,0.202899,0.000000,0.194265,0.537947,0.660
7,0.146588,0.091660,0.542664,0.632549,0.006689,1.000000,0.000000,0.002864,0.000
8,0.439764,0.007432,0.655003,0.770210,0.212932,0.925406,0.024014,0.001432,0.000
9,1.000000,0.332783,1.000000,1.000000,0.433668,0.816624,0.040143,0.000000,0.005


In [24]:
scaled_csv_df = pd.concat([csv_df['날짜'], scaled_df], axis=1)
scaled_csv_df = scaled_csv_df.set_index('날짜')
scaled_csv_df

,ROE,영업이익률,부채비율,EPS,순이익률,BPS,종가,PER,PBR
날짜,,,,,,,,,
20221231,0.666386,0.189926,0.000000,0.025159,1.000000,0.021155,0.539427,0.320764,0.530
20211231,0.427127,0.530140,0.829343,0.013448,0.538462,0.014930,0.951613,0.640095,0.875
20201231,0.101095,0.260116,0.654389,0.003503,0.190635,0.011480,1.000000,1.000000,1.000
20191231,0.000000,0.000000,0.473297,0.000036,0.000000,0.009884,0.548387,0.834368,0.720
20181231,0.816344,1.000000,0.648250,0.016961,0.975474,0.007997,0.241935,0.279236,0.525
20171231,0.923336,0.853840,0.875998,0.014578,0.910814,0.003163,0.461649,0.399045,0.825
20161231,0.270430,0.201486,0.580724,0.000000,0.202899,0.000000,0.194265,0.537947,0.660
20151231,0.146588,0.091660,0.542664,0.632549,0.006689,1.000000,0.000000,0.002864,0.000
20141231,0.439764,0.007432,0.655003,0.770210,0.212932,0.925406,0.024014,0.001432,0.000


In [25]:
#입력데이터, 정답데이터 정의
feature_cols = ['ROE','영업이익률','부채비율','EPS','순이익률','BPS','PER','PBR'] 
label_cols = ['종가'] 

#입력, 정답데이터 프레임
feature_df = scaled_csv_df[feature_cols]
label_df = scaled_csv_df[label_cols]

print(feature_df)
print(label_df)

feature_np = feature_df.to_numpy()
label_np = label_df.to_numpy()

               ROE     영업이익률      부채비율       EPS      순이익률       BPS   
날짜                                                                     
20221231  0.666386  0.189926  0.000000  0.025159  1.000000  0.021155  \
20211231  0.427127  0.530140  0.829343  0.013448  0.538462  0.014930   
20201231  0.101095  0.260116  0.654389  0.003503  0.190635  0.011480   
20191231  0.000000  0.000000  0.473297  0.000036  0.000000  0.009884   
20181231  0.816344  1.000000  0.648250  0.016961  0.975474  0.007997   
20171231  0.923336  0.853840  0.875998  0.014578  0.910814  0.003163   
20161231  0.270430  0.201486  0.580724  0.000000  0.202899  0.000000   
20151231  0.146588  0.091660  0.542664  0.632549  0.006689  1.000000   
20141231  0.439764  0.007432  0.655003  0.770210  0.212932  0.925406   
20131231  1.000000  0.332783  1.000000  1.000000  0.433668  0.816624   

               PER    PBR  
날짜                         
20221231  0.320764  0.530  
20211231  0.640095  0.875  
20201231  1.000000  1.0

In [26]:
window_size = 2   #40개의 입력데이터를 이용해 바로 다음 값에 오는 Close 값을 예측

def make_sequence_dataset(feature, label, window_size):

  feature_list = []   #생성될 feature list
  label_list = []     #생성될 label list

  for i in range(len(feature)-window_size):

    feature_list.append(feature[i:i+window_size])
    label_list.append(label[i+window_size])

  return np.array(feature_list), np.array(label_list)


X, y = make_sequence_dataset(feature_np, label_np, window_size)

print(X.shape, y.shape)

(8, 2, 8) (8, 1)


In [27]:
split = int(len(feature_df)*0.7) #테스트 데이터로 분리

X_train = X[0:split]
y_train = y[0:split]

X_test = X[split:]
y_test = y[split:]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(7, 2, 8) (7, 1)
(1, 2, 8) (1, 1)


In [28]:
#LSTM 모델 구축

model = Sequential()

#LSTM layer
model.add(LSTM(128,
               activation = 'sigmoid',
               input_shape = X_train[0].shape))   #input_shape = (2,8)
#출력층
model.add(Dense(1, activation='linear'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128)               70144     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 70,273
Trainable params: 70,273
Non-trainable params: 0
_________________________________________________________________


In [29]:
# 모델 컴파일
model.compile(loss='mse', optimizer='adam', metrics=['mae'])  # 옵티마이저와 손실 함수, 평가지표 설정

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32)

# 주가 예측
predictions = model.predict(X_test)

Epoch 1/50
1/1 [==============================] - 1s 738ms/step - loss: 0.3722 - mae: 0.5189
Epoch 2/50
1/1 [==============================] - 0s 1ms/step - loss: 0.2463 - mae: 0.3795
Epoch 3/50
1/1 [==============================] - 0s 0s/step - loss: 0.1608 - mae: 0.2914
Epoch 4/50
1/1 [==============================] - 0s 12ms/step - loss: 0.1138 - mae: 0.2527
Epoch 5/50
1/1 [==============================] - 0s 0s/step - loss: 0.1003 - mae: 0.2642
Epoch 6/50
1/1 [==============================] - 0s 10ms/step - loss: 0.1103 - mae: 0.2814
Epoch 7/50
1/1 [==============================] - 0s 0s/step - loss: 0.1304 - mae: 0.3124
Epoch 8/50
1/1 [==============================] - 0s 8ms/step - loss: 0.1480 - mae: 0.3398
Epoch 9/50
1/1 [==============================] - 0s 0s/step - loss: 0.1559 - mae: 0.3525
Epoch 10/50
1/1 [==============================] - 0s 9ms/step - loss: 0.1532 - mae: 0.3488
Epoch 11/50
1/1 [==============================] - 0s 3ms/step - loss: 0.1425 - mae: 0.33

In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# MAE, RMSE, R-Squared 계산
mae = mean_absolute_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
#r2 = r2_score(y_test, predictions)

# 결과 출력
print("MAE: {:.5f}".format(mae))
print("RMSE: {:.5f}".format(rmse))
#print("R-Squared: {:.5f}".format(r2))  

##r2_score를 사용하기에는 샘플 수도 적고, 시계열 데이터 특성상 안 맞을 수도 있을 것 같음.
##지금은 테스트 샘플이 2개 이하라서 nan 값이 나오고, 3개로 늘려봤을 때도 -5000이하의 값이 나온다...

MAE: 0.13411
RMSE: 0.13411


In [ ]:
#종가 데이터만 모두 불러오고, 나머지 데이터는 그 해 년도를 그대로 적용하여 사용 -> 할 수 있을 것 같음